# pγ → ΛK⁺π⁰ amplitude model

PWA study on $p \gamma \to \Lambda K^+ \pi^0$.
We formulate the helicity amplitude model symbolically using `AmpForm` here.

In [ ]:
from __future__ import annotations

import logging
import os
import warnings
from collections import defaultdict
from fractions import Fraction

import ampform
import graphviz
import jax
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qrules
from ampform.dynamics.builder import RelativisticBreitWignerBuilder
from ampform.io import aslatex, improve_latex_rendering
from IPython.display import Markdown, Math, display
from qrules.particle import Particle, Spin, create_particle, load_pdg
from tensorwaves.data import (
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
)
from tensorwaves.function.sympy import create_parametrized_function

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

improve_latex_rendering()
particle_db = load_pdg()

## Decay definition

### Particle definitions

In [ ]:
particles = ["Lambda", "K+", "pi0", "gamma", "p"]
latex_particle_names = {
    "Lambda": r"$\Lambda$",
    "K+": r"$K^+$",
    "pi0": r"$\pi^0$",
    "gamma": r"$\gamma$",
    "p": "p",
}


def format_parity(parity: int | None) -> str:
    if parity is None:
        return ""
    if parity == -1:
        return "-"
    if parity == 1:
        return "+"
    return str(parity)


def format_fraction(value: Fraction | None) -> str | None:
    if value is None:
        return None
    if value.denominator == 1:
        return str(value.numerator)
    return rf"\frac{{{value.numerator}}}{{{value.denominator}}}"


def format_fraction_isospin3(value: Fraction | None) -> str | None:
    if value is None:
        return None
    if value.denominator == 1:
        return str(value.numerator)
    return rf"$\frac{{{value.numerator}}}{{{value.denominator}}}$"


def particle_extract(name: str) -> dict:
    particle = particle_db[name]

    isospin_magnitude_value = (
        Fraction(particle.isospin.magnitude) if particle.isospin is not None else None
    )
    isospin_magnitude_latex = (
        format_fraction(isospin_magnitude_value)
        if isospin_magnitude_value is not None
        else "0"
        if particle.isospin and particle.isospin.magnitude == 0
        else "None"
    )

    isospin_projection_value = (
        Fraction(particle.isospin.projection) if particle.isospin is not None else None
    )
    isospin_projection_latex = (
        format_fraction_isospin3(isospin_projection_value)
        if isospin_projection_value is not None
        else "0"
        if particle.isospin and particle.isospin.projection == 0
        else "None"
    )

    spin_fraction_value = Fraction(particle.spin) if particle.spin is not None else None
    spin_latex = (
        format_fraction(spin_fraction_value)
        if spin_fraction_value is not None
        else "0"
        if particle.spin == 0
        else "None"
    )

    if particle.isospin is not None:
        jpc_ig = rf"${spin_latex}^{{{format_parity(particle.parity)}{format_parity(particle.c_parity)}}} \ ({isospin_magnitude_latex}^{{{format_parity(particle.g_parity)}}})$"
    else:
        jpc_ig = rf"${spin_latex}^{{{format_parity(particle.parity)}{format_parity(particle.c_parity)}}}$"

    return {
        "Name": particle.name,
        "Latex": particle.latex,
        "PID": str(particle.pid),
        r"$J^{PC} (I^G)$": jpc_ig,
        r"$I_3$": isospin_projection_latex,
        r"$M$": str(particle.mass),
        r"$\Gamma$": str(particle.width),
        r"$Q$": str(particle.charge),
        r"$S$": str(particle.strangeness),
        r"$B$": str(particle.baryon_number),
    }


info = [particle_extract(name) for name in particles]
particle_dataframe = pd.DataFrame(info)

particle_dataframe_transposed = particle_dataframe.transpose().reset_index()
particle_dataframe_transposed.columns = ["", *particles]
particle_dataframe_transposed.columns = [
    "",
    *[f"${latex_particle_names.get(name, name)}$" for name in particles],
]

particle_dataframe_transposed.style.hide(axis="index")

In [ ]:
def generate_markdown_table(particles):
    headers = ["Attribute"] + [
        latex_particle_names.get(name, name) for name in particles
    ]
    rows = [headers]

    info = [particle_extract(name) for name in particles]
    for key in info[0]:
        row = [key] + [particle[key] for particle in info]
        rows.append(row)

    table_md = "| " + " | ".join(headers) + " |\n"
    table_md += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    for row in rows[1:]:
        table_md += "| " + " | ".join(row) + " |\n"

    return table_md


particles = ["Lambda", "K+", "pi0", "gamma", "p"]
markdown_table = generate_markdown_table(particles)
display(Markdown(markdown_table))

In [ ]:
def generate_markdown_table(particles):
    headers = ["Particle", *particle_extract(particles[0]).keys()]
    rows = []

    for name in particles:
        row = [latex_particle_names.get(name, name), *particle_extract(name).values()]
        rows.append(row)

    table_md = "| " + " | ".join(headers) + " |\n"
    table_md += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    for row in rows:
        table_md += "| " + " | ".join(row) + " |\n"

    return table_md


particles = ["Lambda", "K+", "pi0", "gamma", "p"]
markdown_table = generate_markdown_table(particles)
display(Markdown(markdown_table))

In the table above, PID is the PDG ID from PDG particle numbering scheme,
$J$ is the spin, $P$ is the parity, $C$ is the C parity, $I$ is the isospin (magnitude), $G$ is the G parity.
$I_3$ is the isospin projection (or the 3rd component),
$M$ is the mass,
$\Gamma$ is the width,
$Q$ is the charge,
$S$ is the strangeness number,
and $B$ is the baryon number.

### Initial state definition

Mass for $p \gamma$ system

In [ ]:
E_lab_gamma = 8.5
m_proton = 0.938
m_0 = np.sqrt(2 * E_lab_gamma * m_proton + m_proton**2)
m_eta = 0.548
m_pi = 0.135
m_0

Add custom particle $p \gamma$

In [ ]:
pgamma1 = Particle(
    name="pgamma1",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=m_0,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma2 = create_particle(
    template_particle=pgamma1,
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    pid=pgamma1.pid + 1,
)
particle_db.update([pgamma1, pgamma2])

### Generate transitions

For simplicity, we use the initial state $p \gamma$ (with spin-$\frac{1}{2}$), 
and set the allowed interaction type to be strong only,
the formalism is selected to be helicity formalism instead of canonical.

:::{seealso}
[Helicity versus canonical](https://ampform.readthedocs.io/stable/usage/helicity/formalism.html)
:::

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("pgamma1"),
    final_state=["Lambda", "K+", "pi0"],
    allowed_interaction_types=["strong"],
    formalism="helicity",
    particle_db=particle_db,
    max_angular_momentum=4,
    max_spin_magnitude=4,
    mass_conservation_factor=0,
)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

## Formulate amplitude model

In [ ]:
model_builder = ampform.get_builder(reaction)
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = 0, 1, 2
bw_builder = RelativisticBreitWignerBuilder(
    energy_dependent_width=False,
    form_factor=False,
)
for name in reaction.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, bw_builder)
model = model_builder.formulate()

In [ ]:
model.intensity

The first term in the amplitude model:

In [ ]:
(symbol, expr), *_ = model.amplitudes.items()
Math(aslatex({symbol: expr}, terms_per_line=1))

In [ ]:
Math(aslatex(model.parameter_defaults))

In [ ]:
Math(aslatex(model.kinematic_variables))

## Visualization

In [ ]:
unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="jax",
)

In [ ]:
phsp_event = 500_000
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
phsp_momenta = phsp_generator.generate(phsp_event, rng)

In [ ]:
helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables,
    backend="jax",
)
phsp = helicity_transformer(phsp_momenta)

In [ ]:
%config InlineBackend.figure_formats = ['png']
fig, ax = plt.subplots(dpi=200)
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
    weights=intensity_func(phsp),
)
ax.set_title("Model-weighted Phase space Dalitz Plot")
ax.set_xlabel(R"$m^2(\Lambda K^+)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(K^+ \pi^0)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
resonances = defaultdict(set)
for transition in reaction.transitions:
    topology = transition.topology
    top_decay_products = topology.get_edge_ids_outgoing_from_node(0)
    (resonance_id, resonance), *_ = transition.intermediate_states.items()
    recoil_id, *_ = top_decay_products - {resonance_id}
    resonances[recoil_id].add(resonance.particle)
resonances = {k: sorted(v, key=lambda p: p.mass) for k, v in resonances.items()}
{k: [p.name for p in v] for k, v in resonances.items()}

In [ ]:
%config InlineBackend.figure_formats = ['svg']
fig, axes = plt.subplots(figsize=(14, 4), ncols=3, sharey=True)
ax1, ax2, ax3 = axes

max_value = 0
color_id = 0
intensities = intensity_func(phsp)
for recoil_id, ax in enumerate(axes):
    decay_products = sorted({0, 1, 2} - {recoil_id})
    key = f"m_{''.join(str(i) for i in decay_products)}"
    bin_values, bin_edges = jax.numpy.histogram(
        phsp[key].real,
        bins=100,
        density=True,
        weights=intensities,
    )
    max_value = max(max_value, bin_values.max())
    ax.fill_between(
        bin_edges[:-1],
        bin_values,
        alpha=0.5,
        step="pre",
    )
    ax.set_ylim(0, None)
    for resonance in resonances[recoil_id]:
        ax.axvline(
            resonance.mass,
            c=f"C{color_id}",
            ls="dotted",
            label=resonance.name,
        )
        color_id += 1

        product_latex = " ".join(
            [reaction.final_state[i].latex for i in decay_products],
        )
        ax.set_xlabel(f"$m({product_latex})$ [GeV]")

        if recoil_id == 0:
            ax.legend(fontsize="small", loc="upper right")
        elif recoil_id == 1:
            ax.legend(fontsize="small", loc="upper center", bbox_to_anchor=(0.6, 1.0))
        else:
            ax.legend(fontsize="small", loc="upper center")
for ax in axes:
    ax.set_ylim(0, max_value * 1.1)

fig.tight_layout()
plt.show(fig)